In [1]:
import os
import json
import sys
sys.path.append('..')

import fire
import numpy as np
import tensorflow as tf
import matplotlib.pylab as plt

import encoder
import model
import sample
from viz import attention

In [2]:
%%javascript
require.config({
  paths: {
      d3: '//cdnjs.cloudflare.com/ajax/libs/d3/3.4.8/d3.min'
  }
});

<IPython.core.display.Javascript object>

In [3]:
%matplotlib notebook

In [4]:
seed=42
model_name='117M'
batch_size=1
np.random.seed(seed)
tf.set_random_seed(seed)

In [5]:
enc = encoder.get_encoder(model_name)
hparams = model.default_hparams()
with open(os.path.join('../pretrained', model_name, 'hparams.json')) as f:
    hparams.override_from_dict(json.load(f))

length = hparams.n_ctx // 2

In [6]:
sess = tf.Session()
context = tf.placeholder(tf.int32, [batch_size, None])
# output = sample.sample_sequence(
#             hparams=hparams, length=length,
#             context=context,
#             batch_size=batch_size
#         )
lm_output = model.model(hparams=hparams, X=context, reuse=tf.AUTO_REUSE)

In [7]:
saver = tf.train.Saver()
ckpt = tf.train.latest_checkpoint(os.path.join('../pretrained', model_name))
saver.restore(sess, ckpt)

INFO:tensorflow:Restoring parameters from ../pretrained/117M/model.ckpt


In [24]:
token_ids, tokens = enc.encode_with_tokens("When Wilhelmina later finds out that MODE legally belongs to Claire Meade, Bradford's wife, she visits her at the hospital.")

In [25]:
out = sess.run(lm_output, feed_dict={
                    context: [token_ids for _ in range(batch_size)]
                })

In [26]:
out['att_probs'].shape

(1, 12, 12, 29, 29)

In [27]:
# plt.imshow(out['att_probs'][0,10,10,:,:], interpolation='nearest', extent=(0.5,10.5,0.5,10.5))
# plt.colorbar()
# plt.show()

In [28]:
tokens

['When',
 'ĠWil',
 'hel',
 'min',
 'a',
 'Ġlater',
 'Ġfinds',
 'Ġout',
 'Ġthat',
 'ĠM',
 'ODE',
 'Ġlegally',
 'Ġbelongs',
 'Ġto',
 'ĠClaire',
 'ĠMe',
 'ade',
 ',',
 'ĠBradford',
 "'s",
 'Ġwife',
 ',',
 'Ġshe',
 'Ġvisits',
 'Ġher',
 'Ġat',
 'Ġthe',
 'Ġhospital',
 '.']

In [34]:
[tokens[0]] + [tok[1:] if tok[0] == '\u0120' else '##'+tok for tok in tokens[1:]]

['When',
 'Wil',
 '##hel',
 '##min',
 '##a',
 'later',
 'finds',
 'out',
 'that',
 'M',
 '##ODE',
 'legally',
 'belongs',
 'to',
 'Claire',
 'Me',
 '##ade',
 '##,',
 'Bradford',
 "##'s",
 'wife',
 '##,',
 'she',
 'visits',
 'her',
 'at',
 'the',
 'hospital',
 '##.']

In [29]:
attentions = []
for _id, layer in enumerate(out['att_probs'][0]):
    attentions.append({
        'index': _id,
        'values': layer
    })

In [30]:
len(attentions)

12

In [31]:
attentions[0]['values'].shape

(12, 29, 29)

In [32]:
attention.show(tokens, attentions)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>